In [1]:
import os, sys
import pandas as pd
import re

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW
reload(RTCW)

db_name = 'rtcw_db.db'
demos_folder = 'demos'
cwd = os.getcwd()
demos_path = os.path.join(cwd, '../', demos_folder)

exe_name = 'Anders.Gaming.LibTech3.exe'
root_path = 'C:\\Users\\Jelle\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path

In [2]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

parsed 100 matches
parsed 200 matches
parsed 300 matches
parsed 400 matches
finished parsing all matches!


In [3]:
#load obituaries in a df
import sqlite3_connector
reload(sqlite3_connector)
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [25]:
obituary_df = db.get_table_as_df(table_name= 'obituary')
demo_df = db.get_table_as_df(table_name= 'demo')
player_df = db.get_table_as_df(table_name= 'player')
chatmessages_df = db.get_table_as_df(table_name= 'chatmessage')
chatmessages_df = RTCW.add_match_data(chatmessages_df, player_df, demos_dct, what_df = 'chatmessages_df')
chatmessages_df = RTCW.feature_extraction_chat(chatmessages_df)

obituary table fetched in 0.44 seconds
demo table fetched in 0.01 seconds
player table fetched in 0.05 seconds
chatmessage table fetched in 0.64 seconds


In [29]:
docs_df = chatmessages_df.loc[chatmessages_df['DocsAll'] | chatmessages_df['TimelimitHit']]

In [109]:
def get_docruns(chatmessages_df, min_docrun_length = None, max_timeleft = None, docs_succesful = None, min_docs_lost = None, verbose = True):
    
    #empty lists that we will populate and use to make final df
    pd_md5 = []
    pd_start_docrun_dwtime = []
    pd_end_docrun_dwtime = []
    pd_start_docrun_secsleft = []
    pd_end_docrun_secsleft = []
    pd_docrun_duration = []
    pd_times_docs_lost = []
    pd_won_round = []
    pd_demo_name = []
    pd_match_name = []
    
    #helper variables for verbose
    counter = 0
    total_demos = obituary_df.szMd5.nunique()
    
    for demo in chatmessages_df.szMd5.unique():
        #evt hier nog cutten op timelimit hit of docsevents zoals cell hierboven voor speed
        df_demo = chatmessages_df.loc[(chatmessages_df['szMd5'] == demo) & (chatmessages_df['InMatch'])]
        demo_name = df_demo.demoName.unique()
        match_name = df_demo.matchName.unique()
        counter += 1
        
        arr = df_demo.as_matrix(columns = ['DocsEvents', 'TimelimitHit', 'dwTime', 'SecondsLeftInRound'])
        taken_docs_bool = False
        docs_lost_times = 0
        
        for row in range(len(arr)):
            chat = arr[row, :]
            
            #taken docs
            if chat[0] == 1:
                if taken_docs_bool == False: 
                    start_dwtime = chat[2]
                    start_timeleft = chat[3]
                    taken_docs_bool = True
                else:
                    docs_lost_times += 1

            #transmitted docs
            if chat[0] == 2:
                end_dwtime = chat[2]
                end_timeleft = chat[3]
                taken_docs_bool = False
                docs_succes = True
                
                pd_md5.append(demo)
                pd_start_docrun_dwtime.append(start_dwtime)
                pd_end_docrun_dwtime.append(end_dwtime)
                pd_start_docrun_secsleft.append(start_timeleft)
                pd_end_docrun_secsleft.append(end_timeleft)
                pd_docrun_duration.append(start_timeleft - end_timeleft)
                pd_times_docs_lost.append(docs_lost_times)
                pd_won_round.append(docs_succes)
                pd_demo_name.append(demo_name[0])
                pd_match_name.append(match_name[0])
            
                docs_lost_times = 0
                
            #returned docs / timelimit hit
            if ((chat[0] == 0 or chat[1] == 1) and (taken_docs_bool == True)):
                end_dwtime = chat[2]
                end_timeleft = chat[3]
                taken_docs_bool = False
                docs_succes = False
                
                pd_md5.append(demo)
                pd_start_docrun_dwtime.append(start_dwtime)
                pd_end_docrun_dwtime.append(end_dwtime)
                pd_start_docrun_secsleft.append(start_timeleft)
                pd_end_docrun_secsleft.append(end_timeleft)
                pd_docrun_duration.append(start_timeleft - end_timeleft)
                pd_times_docs_lost.append(docs_lost_times)
                pd_won_round.append(docs_succes)
                pd_demo_name.append(demo_name[0])
                pd_match_name.append(match_name[0])
                
                docs_lost_times = 0
                
            #if demo stopped before end of docrun
            
            
        #verbose shizzle
        if counter % 100 == 0:
            if verbose:
                print 'scanned ' + str(counter) + ' demos of ' + str(total_demos) + ' demos in total' 
    
    print 'all done!'
    
    
    #make final dataframe where 1 row is a spree with all the necessary info
    df_docs = pd.DataFrame(
    {'md5': pd_md5,
     'start_dwtime': pd_start_docrun_dwtime,
     'end_dwtime': pd_end_docrun_dwtime,
     'start_secsleft': pd_start_docrun_secsleft,
     'end_secsleft': pd_end_docrun_secsleft,
     'duration': pd_docrun_duration,
     'times_lost_docs': pd_times_docs_lost,
     'won_round': pd_won_round,
     'demo': pd_demo_name,
     'match': pd_match_name,
    })
    
    
    
    if min_docrun_length != None:
        df_docs = df_docs.loc[df_docs['duration'] >= min_docrun_length]
        
    if max_timeleft != None:
        df_docs = df_docs.loc[df_docs['end_secsleft'] <= max_timeleft]
        
    if docs_succesful != None:
        df_docs = df_docs.loc[df_docs['won_round'] == docs_succesful]
        
    if min_docs_lost != None:
        df_docs = df_docs.loc[df_docs['times_lost_docs'] >= min_docs_lost]
        
    return df_docs    

In [110]:
chattest = chatmessages_df.loc[chatmessages_df['szMd5'] == '2c60f8f66f42642e8e3c12d3e45d4500']

In [127]:
reload(RTCW)
res = RTCW.get_docruns(chatmessages_df, min_docrun_length = 10, max_timeleft = 3, docs_succesful = True, min_docs_lost = 0, verbose = True)
res[['start', 'end', 'start_secsleft', 'end_secsleft', 'duration', 'times_lost_docs', 'won_round', 'md5']]

scanned 100 demos of 1270 demos in total
scanned 200 demos of 1270 demos in total
scanned 300 demos of 1270 demos in total
scanned 400 demos of 1270 demos in total
scanned 500 demos of 1270 demos in total
scanned 600 demos of 1270 demos in total
scanned 700 demos of 1270 demos in total
scanned 800 demos of 1270 demos in total
scanned 900 demos of 1270 demos in total
scanned 1000 demos of 1270 demos in total
scanned 1100 demos of 1270 demos in total
scanned 1200 demos of 1270 demos in total
all done!


,start,end,start_secsleft,end_secsleft,duration,times_lost_docs,won_round,md5
1438,56845850,56886250,41,0,41,2,True,e759ddacddba9d350a13e5d5a73bc39b
1505,3112700,3133500,24,3,21,0,True,5d3fb329dbf49bc6140a69c8dd99e0c3
2610,57535300,57589100,55,1,54,2,True,54a794adb5066aa16cda965c9928d2a8
2614,600750594,600812194,62,0,62,4,True,8b2bcc7d5ad48a8a608606476191a88f
2770,1771757,1825580,54,0,54,0,True,e1d78ba4544cd69790269cbc796e6884
2783,5266406,5316830,51,0,51,1,True,1fac5e7a80f23b5e42ce07c7838de300
2915,80932320,80970963,40,1,39,0,True,f2b659cf9e64cf25243ca3ee5c699999
3138,17085683,17105933,20,0,20,2,True,4fed80be73e03afd867187945f7a9482


In [138]:
reload(RTCW)
RTCW.cut_demos(root_path, demos_dct, res, demo_type = 'docs', offset_start = 8, offset_end = 8, transform_to_dm_60 = True, demo_folder_name= 'demos')

demo                ECX_dignitas_vs_emaho_SEMIFINALS_mp_beach4.dm_60
duration                                                          41
end                                                         56886250
end_secsleft                                                       0
match              rtcw_2004.12.12_cb-ec10_dignitas_vs_emaho_semi...
md5                                 e759ddacddba9d350a13e5d5a73bc39b
start                                                       56845850
start_secsleft                                                    41
times_lost_docs                                                    2
won_round                                                       True
Name: 1438, dtype: object
demo                      dt_vs_narf_cal-m.finals.mp_beach.dm_60
duration                                                      21
end                                                      3133500
end_secsleft                                                   3
match              rtcw_

In [ ]:
RTCW.generate_capture_list(df_spree, transform_to_dm_60 = True)

In [123]:
spree = res.iloc[0]
spree['demo'][:-6] + '_' + str(spree['duration']) + '_' + str(spree['start_secsleft']) + '_' + str(spree['end_secsleft']) + '_' + str(spree['won_round']) + '_' + str(spree['start_dwtime']) + spree['demo'][-6:] 

'ECX_dignitas_vs_emaho_SEMIFINALS_mp_beach4_41_41_0_True_56845850.dm_60'